# 6) SageMaker pipeline
In this notebook, we will create a sagemaker pipeline to automate the process of training a model, and updating the deployed endpoint if it perform better on a given test set.  
Also slighty depricated, this documentation helped a lot:
https://docs.aws.amazon.com/sagemaker/latest/dg/define-pipeline.html

In [18]:
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.pipeline import Pipeline
from PIL import Image
import io
import base64
import json
import pprint
import time


session = sagemaker.Session()

bucket = session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = get_execution_role()
print("RoleArn: {}".format(role))

prefix = "capstone-inventory-project"
model_package_group_name = "Capstone_pipeline"

Default Bucket: sagemaker-us-east-1-646714458109
AWS Region: us-east-1
RoleArn: arn:aws:iam::646714458109:role/service-role/AmazonSageMaker-ExecutionRole-20211122T183493


### 1) Define Pipeline Parameters

training_instance_type – The ml.* instance type of the training jobs.

input_data – The Amazon S3 location of the input data.

In [19]:
training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

input_data = ParameterString(
    name="InputData",
    default_value="s3://{}/{}/data".format(bucket, prefix)
)

# We will also define the were our model artefacts and evaluation results will be stored.
model_path = "s3://{}/{}/pipeline_model".format(bucket, prefix)
evaluation_file_path = "s3://{}/{}/current_accuracy.json".format(bucket, prefix)

### 1) Training step

In [20]:
estimator = PyTorch(
    entry_point="pipeline/1_train_pipeline.py",
    role=role,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.m5.xlarge",        
    output_path = model_path,  # The training jobs output (mainly model artefacts) will go there.
    hyperparameters={                                        
        "batch-size": 64,
        "lr": 0.01}
)

In [21]:
step_train = TrainingStep(
    name="Training",
    estimator=estimator,
    inputs=input_data
)

### 2) Evaluation step

In [22]:
image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=region,
    version="1.8",
    py_version="py36",
    instance_type=training_instance_type,
    image_scope="training"
)

In [23]:
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=training_instance_type,
    instance_count=1,
    base_job_name="pipeline-eval",
    role=role,
)

In [24]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)
last_report = PropertyFile(
    name="LastReport",
    output_name="evaluation",
    path="current_accuracy.json"
)

step_eval = ProcessingStep(
    name="Evaluation",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source="s3://sagemaker-us-east-1-646714458109/capstone-inventory-project/data/test/",
            destination="/opt/ml/processing/test"
        ),
        ProcessingInput(
            source=evaluation_file_path,
            destination="/opt/ml/processing/accuracy"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="pipeline/2_evaluation.py",
    property_files=[evaluation_report, last_report]
)

### 3) Test step

In [25]:
step_test = ProcessingStep(
    name="Test-upload",
    processor=script_eval,
    code="pipeline/test_step.py"
)

### 4) Create model

In [26]:
pytorch_model = PyTorchModel(model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts, 
                             role=role, 
                             entry_point='scripts/inference.py',
                             py_version='py3',
                             framework_version='1.5')


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.eia1.medium",
)


step_create_model = CreateModelStep(
    name="Create-model",
    model=pytorch_model,
    inputs=inputs,
)

TypeError: __init__() missing 3 required positional arguments: 'content_types', 'response_types', and 'transform_instances'

### 5) Condition step

In [27]:
# False step
script_else = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="else",
    role=role,
)

step_fail = ProcessingStep(
    name="Too-bad",
    processor=script_else,
    code="pipeline/4_else.py"
)

In [28]:
cond_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name="Evaluation",
        property_file=evaluation_report,
        json_path="test_accuracy"
    ),
    right=JsonGet(
        step_name="Evaluation",
        property_file=last_report,
        json_path="Accuracy_of_deployed_model"
    )
)

In [29]:
step_cond = ConditionStep(
    name="Is-the-new_model-better-than-the-one-deployed",
    conditions=[cond_gte],
    if_steps=[step_test],  # step_register
    else_steps=[step_fail]
)

### 4) Create model

### 5) Pipeline creation

In [30]:
pipeline_name = "CapstonePipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        training_instance_type,
        input_data
    ],
    steps=[step_train, step_eval, step_cond]
)

In [31]:
json.loads(pipeline.definition())

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-646714458109/capstone-inventory-project/data'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'Training',
   'Type': 'Training',
   'Arguments': {'AlgorithmSpecification': {'TrainingInputMode': 'File',
     'TrainingImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.8-cpu-py36',
     'EnableSageMakerMetricsTimeSeries': True},
    'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-646714458109/capstone-inventory-project/pipeline_model'},
    'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
    'ResourceConfig': {'InstanceCount': 1,
     'InstanceType': 'ml.m5.xlarge',
     'VolumeSizeInGB': 30},
    '

In [32]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:646714458109:pipeline/capstonepipeline',
 'ResponseMetadata': {'RequestId': 'c7b96eae-eb7d-44e0-91ad-0fb899211ce1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c7b96eae-eb7d-44e0-91ad-0fb899211ce1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Thu, 20 Jan 2022 16:14:05 GMT'},
  'RetryAttempts': 0}}

In [17]:
#pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:646714458109:pipeline/capstonepipeline',
 'ResponseMetadata': {'RequestId': '66a5bfce-5385-4b8d-86ac-006da983d63d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '66a5bfce-5385-4b8d-86ac-006da983d63d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '84',
   'date': 'Thu, 20 Jan 2022 16:13:19 GMT'},
  'RetryAttempts': 0}}

In [35]:
#execution = pipeline.start()

In [45]:
execution.list_steps()

[{'StepName': 'Evaluation',
  'StartTime': datetime.datetime(2022, 1, 20, 14, 7, 16, 413000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 1, 20, 14, 13, 23, 770000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'AttemptCount': 0,
  'FailureReason': 'ClientError: Cannot access S3 key.',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:646714458109:processing-job/pipelines-1kfl6nqdpjnj-evaluation-2dvfanug7b'}}},
 {'StepName': 'Training',
  'StartTime': datetime.datetime(2022, 1, 20, 13, 58, 57, 172000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 1, 20, 14, 7, 15, 617000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:646714458109:training-job/pipelines-1kfl6nqdpjnj-training-4sxzkheorc'}}}]

In [42]:
execution.list_steps()[0]["Metadata"]

{'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:646714458109:processing-job/pipelines-1kfl6nqdpjnj-evaluation-2dvfanug7b'}}